<a href="https://colab.research.google.com/github/YunshuoTian/nlp/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np

ModuleNotFoundError: No module named 'datasets'

In [1]:
# load a dataset from hugginface
dataset = load_dataset("akoksal/LongForm")

NameError: name 'load_dataset' is not defined

In [4]:
# how the data look like
dataset['train'][101]

{'input': '"Who was Nyman Levin and what were his family connections?"',
 'output': 'Doctor Nyman Levin (born 17 February 1906, London; d. 25 January 1965, London) was a British physicist who was Director of the Atomic Weapons Research Establishment from 1959 to 1965.\nNyman Levin was the son of Leibisch Levin and Annie Levene; there was one brother, Barnett, and several half-siblings, Annie, Fanny, Solomon and Mary. He married Dora and the couple had three children.',
 'source': 'Wikipedia',
 'subset': 'search-engine'}

In [5]:
# load tokenizer and model
model_name = 'bert-base-uncased'
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
print(tokenizer.vocab_size)
print(tokenizer.model_max_length)

30522
512


In [6]:
# start preprocessing
def preprocess_data(examples):
  return tokenizer(
      examples['input'],
      padding=True,
      truncation=True,
      return_tensors='tf'
  )

In [9]:
# get the first 3000 data in dataset for training

train_dataset = dataset["train"].select(range(3000))
eval_dataset = dataset["validation"]
tokenized_trainset = train_dataset.map(preprocess_data, batched=True, remove_columns=dataset["train"].column_names)
tokenized_evalset = eval_dataset.map(preprocess_data, batched=True, remove_columns=dataset["validation"].column_names)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [10]:
# define training args
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [11]:
from transformers import default_data_collator

data_collator = default_data_collator

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_trainset,
    eval_dataset=tokenized_evalset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()